<a href="https://colab.research.google.com/github/githubuserpk/gcp/blob/master/genai/bigquery_authentication_method1_using_auth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install vertex ai
!pip install google-cloud-aiplatform



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


In [2]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [3]:
#install vertex ai
!pip install google-cloud-aiplatform

#install vertex ai
!pip install google-cloud-aiplatform

#REGION='us-central1'
PROJECT_ID='pkdeltaai-01'

In [6]:
import vertexai
vertexai.init(project="pkdeltaai-01") #, location="us-central1")

from google.cloud import bigquery
import pandas as pd

#This function takes a sql query string and runs it on bigquery

def run_bq_query(sql):

    # Create BQ client
    bq_client = bigquery.Client(project = PROJECT_ID) # location=REGION)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True,
                                         use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql,
                                    job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [7]:
# define list of programming language tags we want to query

#language_list = ["python", "html", "r", "css"]
language_list = ["python"]

so_df = pd.DataFrame()

for language in language_list:

    print(f"generating {language} dataframe")

    query = f"""
    SELECT
        CONCAT(q.title, q.body) as input_text,
        a.body AS output_text
    FROM
        `bigquery-public-data.stackoverflow.posts_questions` q
    JOIN
        `bigquery-public-data.stackoverflow.posts_answers` a
    ON
        q.accepted_answer_id = a.id
    WHERE
        q.accepted_answer_id IS NOT NULL AND
        REGEXP_CONTAINS(q.tags, "python") AND
        a.creation_date >= "2020-01-01"
    LIMIT
        500
    """


    language_df = run_bq_query(query)
    language_df["category"] = language
    so_df = pd.concat([so_df, language_df],
                      ignore_index = True)


generating python dataframe
Finished job_id: 399a7a9b-6792-4140-93d6-d857ea3dd4ba
